In [1]:
import os
import cv2
import json
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw
import re  

def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()

def extract_roi_from_polygon(image, polygon_points, target_width, target_height):
    polygon_points_tuples = [(x, y) for [x, y] in polygon_points]

    mask = Image.new('L', (image.shape[1], image.shape[0]), 0)
    ImageDraw.Draw(mask).polygon(polygon_points_tuples, outline=1, fill=1)
    mask = np.array(mask)
    roi = image * mask[:, :, np.newaxis]

    roi = cv2.resize(roi, (target_width, target_height), interpolation=cv2.INTER_AREA)
    
    return roi

root_directory = r'C:\Users\acer\OneDrive\Desktop\annotator\Task 2_Bhat'
all_features = []
all_labels = []
all_annotators = []
all_image_numbers = []  

for subfolder in range(1, 14):
    folder_name = f's{subfolder}'
    folder_path = os.path.join(root_directory, folder_name)

    if os.path.exists(folder_path):
        for sub_subfolder in os.listdir(folder_path):
            sub_subfolder_path = os.path.join(folder_path, sub_subfolder)

            if os.path.isdir(sub_subfolder_path):
                for filename in os.listdir(sub_subfolder_path):
                    if filename.endswith('.jpg'):
                        jpg_filepath = os.path.join(sub_subfolder_path, filename)
                        json_filepath = os.path.join(sub_subfolder_path, f"{filename.split('.')[0]}.json")

                        if os.path.exists(json_filepath):
                            with open(json_filepath, 'r') as json_file:
                                data = json.load(json_file)
                                annotations = data.get('shapes', [])

                            image = cv2.imread(jpg_filepath)
                            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                            # Extract image number from filename using regex specific to your filename format
                            image_number = re.findall(r's(\d+)', filename)  # This regex finds digits after 's'
                            if image_number:
                                image_number = image_number[0]

                            for annotation in annotations:
                                label = annotation['label']
                                polygon_points = annotation['points']

                                roi = extract_roi_from_polygon(image, polygon_points, 100, 100)
                                rgb_values = extract_rgb_values(roi)

                                all_features.append(rgb_values)
                                all_labels.append(label)
                                all_annotators.append(root_directory)
                                all_image_numbers.append(image_number)  # Store the extracted image number

# Create a DataFrame including the new image_number column
df1 = pd.DataFrame({
    'labels': all_labels,
    'features': all_features,
    'annotator': all_annotators,
    'image_number': all_image_numbers  # New column for image numbers
})


In [4]:
df1[:50]

,labels,features,annotator,image_number
0,Mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,1
1,Mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,1
2,Mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,1
3,Animals,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,1
4,Animals,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,1
5,Human,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,10
6,Human,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,11
7,Hut,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,11
8,Hut,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,11
9,Hut,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,11


In [44]:
import os
import cv2
import json
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw

def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()

def extract_roi_from_polygon(image, polygon_points, target_width, target_height):
    polygon_points_tuples = [(x, y) for [x, y] in polygon_points]

    mask = Image.new('L', (image.shape[1], image.shape[0]), 0)
    ImageDraw.Draw(mask).polygon(polygon_points_tuples, outline=1, fill=1)
    mask = np.array(mask)
    roi = image * mask[:, :, np.newaxis]

    roi = cv2.resize(roi, (target_width, target_height), interpolation=cv2.INTER_AREA)
    
    return roi

root_directory = r'C:\Users\acer\OneDrive\Desktop\annotator\Task 2_Bhat'
all_features = []
all_labels = []
all_annotators = []
all_image_numbers = []  
all_folders = [] 

for subfolder in range(1, 14):
    folder_name = f's{subfolder}'
    folder_path = os.path.join(root_directory, folder_name)

    if os.path.exists(folder_path):
        for sub_subfolder in os.listdir(folder_path):
            sub_subfolder_path = os.path.join(folder_path, sub_subfolder)

            if os.path.isdir(sub_subfolder_path):
                for filename in os.listdir(sub_subfolder_path):
                    if filename.endswith('.jpg'):
                        jpg_filepath = os.path.join(sub_subfolder_path, filename)
                        json_filepath = os.path.join(sub_subfolder_path, f"{filename.split('.')[0]}.json")

                        if os.path.exists(json_filepath):
                            with open(json_filepath, 'r') as json_file:
                                data = json.load(json_file)
                                annotations = data.get('shapes', [])
                                image_path = data.get('imagePath')  # Extract the image path from JSON

                            image = cv2.imread(jpg_filepath)
                            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                            for annotation in annotations:
                                label = annotation['label']
                                polygon_points = annotation['points']

                                roi = extract_roi_from_polygon(image, polygon_points, 100, 100)
                                rgb_values = extract_rgb_values(roi)

                                all_features.append(rgb_values)
                                all_labels.append(label)
                                all_annotators.append(root_directory)
                                all_image_numbers.append(image_path)  
                                all_folders.append(folder_name) 


df1 = pd.DataFrame({
    'labels': all_labels,
    'features': all_features,
    'annotator': all_annotators,
    'image_number': all_image_numbers, 
    'folder_name': all_folders 
})


In [11]:
df1[:50]

,labels,features,annotator,image_number
0,Mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg
1,Mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg
2,Mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg
3,Animals,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg
4,Animals,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg
5,Human,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s10.jpg
6,Human,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s11.jpg
7,Hut,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s11.jpg
8,Hut,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s11.jpg
9,Hut,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s11.jpg


In [45]:
import os
import cv2
import json
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw


def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()


def extract_roi_from_polygon(image, polygon_points, target_width, target_height):
    polygon_points_tuples = [(x, y) for [x, y] in polygon_points]

    mask = Image.new('L', (image.shape[1], image.shape[0]), 0)
    ImageDraw.Draw(mask).polygon(polygon_points_tuples, outline=1, fill=1)
    mask = np.array(mask)
    roi = image * mask[:, :, np.newaxis]

    
    roi = cv2.resize(roi, (target_width, target_height), interpolation=cv2.INTER_AREA)
    
    return roi

root_directory = r'C:\Users\acer\OneDrive\Desktop\annotator\shuja_t2'


all_features = []
all_labels = []
all_annotators = []
all_image_numbers = []
all_folders = []

for subfolder in range(1, 14):
    folder_name = f's{subfolder}'
    folder_path = os.path.join(root_directory, folder_name)

    
    if os.path.exists(folder_path):
        
        for sub_subfolder in os.listdir(folder_path):
            sub_subfolder_path = os.path.join(folder_path, sub_subfolder)

            
            if os.path.isdir(sub_subfolder_path):
                
                for filename in os.listdir(sub_subfolder_path):
                    
                    if filename.endswith('.jpg'):
                        
                        jpg_filepath = os.path.join(sub_subfolder_path, filename)
                        json_filepath = os.path.join(sub_subfolder_path, f"{filename.split('.')[0]}.json")

                        
                        if os.path.exists(json_filepath):
                            
                            with open(json_filepath, 'r') as json_file:
                                data = json.load(json_file)
                                annotations = data.get('shapes', [])
                                image_path = data.get('imagePath')

                            
                            image = cv2.imread(jpg_filepath)
                            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                            
                            for annotation in annotations:
                                label = annotation['label']
                                polygon_points = annotation['points']

                                roi = extract_roi_from_polygon(image, polygon_points,100,100)
                                rgb_values = extract_rgb_values(roi)

                                all_features.append(rgb_values)
                                all_labels.append(label)
                                all_annotators.append(root_directory) 
                                all_image_numbers.append(image_path)
                                all_folders.append(folder_name)


df2 = pd.DataFrame({
'labels': all_labels,
'features': all_features,
'annotator': all_annotators, 
'image_number': all_image_numbers,
'folder_name': all_folders
})


In [46]:
import os
import cv2
import json
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw


def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()


def extract_roi_from_polygon(image, polygon_points, target_width, target_height):
    polygon_points_tuples = [(x, y) for [x, y] in polygon_points]

    mask = Image.new('L', (image.shape[1], image.shape[0]), 0)
    ImageDraw.Draw(mask).polygon(polygon_points_tuples, outline=1, fill=1)
    mask = np.array(mask)
    roi = image * mask[:, :, np.newaxis]

    
    roi = cv2.resize(roi, (target_width, target_height), interpolation=cv2.INTER_AREA)
    
    return roi

root_directory = r'C:\Users\acer\OneDrive\Desktop\annotator\Solution_Assignment_1_Task_2'

all_features = []
all_labels = []
all_annotators = []
all_image_numbers = []
all_folders = []

for subfolder in range(1, 14):
    folder_name = f's{subfolder}'
    folder_path = os.path.join(root_directory, folder_name)

    
    if os.path.exists(folder_path):
        
        for filename in os.listdir(folder_path):
            
            if filename.endswith('.jpg'):
                
                jpg_filepath = os.path.join(folder_path, filename)
                json_filepath = os.path.join(folder_path, f"{filename.split('.')[0]}.json")

                
                if os.path.exists(json_filepath):
                    
                    with open(json_filepath, 'r') as json_file:
                        data = json.load(json_file)
                        annotations = data.get('shapes', [])
                        image_path = data.get('imagePath')

                    
                    image = cv2.imread(jpg_filepath)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    for annotation in annotations:
                        label = annotation['label']
                        polygon_points = annotation['points']

                        roi = extract_roi_from_polygon(image, polygon_points,100,100)
                        rgb_values = extract_rgb_values(roi)

                        all_features.append(rgb_values)
                        all_labels.append(label)
                        all_annotators.append(root_directory)  
                        all_image_numbers.append(image_path)
                        all_folders.append(folder_name)


df3 = pd.DataFrame({
    'labels': all_labels,
    'features': all_features,
    'annotator': all_annotators,  
    'image_number': all_image_numbers,
    'folder_name': all_folders
})

In [22]:
re.findall(r's[0-9]+.jpg',df3['image_number'].iloc[0])[0]

's1.jpg'

In [47]:
import re
def regex_replace(r):
    return re.findall(r's[0-9]+.jpg',r)[0]

In [48]:
df3['img_number']=df3['image_number'].apply(regex_replace)

In [49]:
df3.drop(columns=['image_number'],inplace=True)

In [50]:
df3.rename(columns={'img_number':'image_number'},inplace=True)

In [51]:
df3

,labels,features,annotator,folder_name,image_number
0,mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Solut...,s1,s1.jpg
1,mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Solut...,s1,s1.jpg
2,mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Solut...,s1,s1.jpg
3,animals,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Solut...,s1,s1.jpg
4,mythological characters,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Solut...,s1,s1.jpg
...,...,...,...,...,...
802,background,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Solut...,s13,s8.jpg
803,background,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Solut...,s13,s8.jpg
804,background,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Solut...,s13,s8.jpg
805,background,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Solut...,s13,s8.jpg


In [52]:
import os
import cv2
import json
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw


def extract_rgb_values(roi):
    rgb_values = roi.reshape(-1, 3) 
    return rgb_values.flatten()

def extract_roi_from_polygon(image, polygon_points, target_width, target_height):
    polygon_points_tuples = [(x, y) for [x, y] in polygon_points]

    mask = Image.new('L', (image.shape[1], image.shape[0]), 0)
    ImageDraw.Draw(mask).polygon(polygon_points_tuples, outline=1, fill=1)
    mask = np.array(mask)
    roi = image * mask[:, :, np.newaxis]

    
    roi = cv2.resize(roi, (target_width, target_height), interpolation=cv2.INTER_AREA)
    
    return roi


root_directory = r'C:\Users\acer\OneDrive\Desktop\annotator\Task 2_Ramchandani'


all_features = []
all_labels = []
all_annotators = []
all_image_numbers = []
all_folders = []

for subfolder in range(1, 14):
    folder_name = str(subfolder)
    folder_path = os.path.join(root_directory, folder_name)

    
    if os.path.exists(folder_path):
        
        for filename in os.listdir(folder_path):
           
            if filename.endswith('.jpg'):
                
                jpg_filepath = os.path.join(folder_path, filename)
                json_filepath = os.path.join(folder_path, f"{filename.split('.')[0]}.json")

                
                if os.path.exists(json_filepath):
                    
                    with open(json_filepath, 'r') as json_file:
                        data = json.load(json_file)
                        annotations = data.get('shapes', [])
                        image_path = data.get('imagePath')

                    
                    image = cv2.imread(jpg_filepath)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    
                    for annotation in annotations:
                        label = annotation['label']
                        polygon_points = annotation['points']
                        
                        roi = extract_roi_from_polygon(image, polygon_points,100,100)
                        rgb_values = extract_rgb_values(roi)

                        all_features.append(rgb_values)
                        all_labels.append(label)
                        all_annotators.append(root_directory)  
                        all_image_numbers.append(image_path)
                        all_folders.append(folder_name)

df4 = pd.DataFrame({
    'labels': all_labels,
    'features': all_features,
    'annotator': all_annotators, 
    'image_number': all_image_numbers,
    'folder_name': all_folders
})

In [37]:
df4['image_number'].iloc[0]

'../../Users/manaliramchandani/Downloads/ProjectData/s1/img/s1.jpg'

In [53]:
df4['image_number']=df4['image_number'].apply(regex_replace)

In [54]:

df4['folder_name']=df4['folder_name'].astype('int')

In [55]:
def change_col_name(r):
    return f's{r}'

In [56]:
df4['folder_name']=df4['folder_name'].apply(change_col_name)

In [81]:
combined_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [59]:
combined_df.to_csv('combined.csv',index=False)

In [86]:
# for each image check if animal is present. if it is take mean of all animal rows and that is the feature row
# if animal is not present take 0 across all features--> acorss all annotators 

In [82]:
def rename_label(s):
    if s=='Animals' or s=='animals' or s=='animal':
        return 'Animal'
    else:
        return 'Not Animal'

In [83]:
combined_df['labels']=combined_df['labels'].apply(rename_label)

In [84]:
combined_df

,labels,features,annotator,image_number,folder_name
0,Not Animal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
1,Not Animal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
2,Not Animal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
3,Animal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
4,Animal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
...,...,...,...,...,...
2659,Not Animal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13
2660,Not Animal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13
2661,Not Animal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13
2662,Not Animal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13


In [37]:
import pandas as pd
import numpy as np

def process_images_with_labels(df):
    
    grouped = df.groupby(['folder_name', 'image_number'])

    processed_features = []
    labels = []

    
    for group_identifier, group in grouped:
        # Check if 'Animal' is present in the group
        if 'Animal' in group['labels'].values:
            # Extract features for rows with label 'Animals'
            animal_features = np.array([np.array(row) for row in group[group['labels'] == 'Animal']['features']])
            # Calculate the mean of features
            mean_features = np.mean(animal_features, axis=0)
            label = 'Animal'  # Since we found at least one 'Animal' label
        else:
            # If 'Animals' not present, create a zero array and label as 'Not Animal'
            mean_features = np.zeros(feature_length)  # feature_length needs to be defined based on your data
            label = 'Not Animal'
        
        # Append the result and the label to the lists
        processed_features.append(mean_features)
        labels.append(label)

    # Return both the features and the labels
    return processed_features, labels

# Apply the function to the DataFrame
resulting_features, resulting_labels = process_images_with_labels(combined_df)

# Create a new DataFrame with the results and labels
result_df = pd.DataFrame({
    'mean_animal_features': resulting_features,
    'label': resulting_labels
})



NameError: name 'feature_length' is not defined

In [128]:
result_df

,mean_animal_features,label
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Not Animal
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Not Animal
...,...,...
101,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
102,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Not Animal
103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
104,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal


In [129]:
def expand_features(row):
    return pd.Series(row['mean_animal_features'])

In [130]:
expanded_df = result_df.apply(expand_features, axis=1)
new_columns = [f"feature {i}" for i in range(1, 30001)]
expanded_df.columns = new_columns
animal_df = pd.concat([result_df, expanded_df], axis=1)

In [131]:
animal_df.drop(columns=['mean_animal_features'],inplace=True)

In [132]:
animal_df

,label,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 29991,feature 29992,feature 29993,feature 29994,feature 29995,feature 29996,feature 29997,feature 29998,feature 29999,feature 30000
0,Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Not Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Not Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,Not Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### mean of animal_df

In [89]:
import pandas as pd
import numpy as np

def process_images_with_labels(df):
    # Group by unique image identifiers
    grouped = df.groupby(['folder_name', 'image_number'])

    # Initialize lists to hold the mean feature arrays, zero arrays, and labels
    processed_features = []
    labels = []
    feature_length=30000

    # Iterate over each group
    for group_identifier, group in grouped:
        # Check if 'Animal' is present in the group
        if 'Animal' in group['labels'].values:
            # Extract features for rows with label 'Animals'
            animal_features = np.array([np.array(row) for row in group[group['labels'] == 'Animal']['features']])
            # Calculate the mean of features
            mean_features = np.mean(animal_features, axis=0)
            label = 'Animal'  # Since we found at least one 'Animal' label
        else:
            # If 'Animals' not present, calculate the mean of 'Not Animal' features
            not_animal_features = np.array([np.array(row) for row in group[group['labels'] == 'Not Animal']['features']])
            if not_animal_features.size > 0:
                mean_features = np.mean(not_animal_features, axis=0)
            else:
                # In case there are no 'Not Animal' rows, we might decide to fill with zeros or handle differently
                mean_features = np.zeros(feature_length)  # feature_length needs to be defined based on your data
            label = 'Not Animal'
        
        # Append the result and the label to the lists
        processed_features.append(mean_features)
        labels.append(label)

    # Return both the features and the labels
    return processed_features, labels

# Example usage with your combined DataFrame
resulting_features, resulting_labels = process_images_with_labels(combined_df)

# Create a new DataFrame with the results and labels
result_df = pd.DataFrame({
    'mean_animal_features': resulting_features,
    'label': resulting_labels
})




In [90]:
result_df

,mean_animal_features,label
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Not Animal
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Not Animal
...,...,...
101,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
102,"[0.2, 0.26666666666666666, 0.4, 2.8, 2.6, 1.8,...",Not Animal
103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal
104,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Animal


In [91]:
def expand_features(row):
    return pd.Series(row['mean_animal_features'])

expanded_df = result_df.apply(expand_features, axis=1)
new_columns = [f"feature {i}" for i in range(1, 30001)]
expanded_df.columns = new_columns
mean_animal_df = pd.concat([result_df, expanded_df], axis=1)

mean_animal_df.drop(columns=['mean_animal_features'],inplace=True)

In [93]:
mean_animal_df.to_csv('mean_animal.csv',index=False)

##### Tree df

In [32]:
combined_df['labels'].value_counts()[:30]

human                      1299
mythological characters     343
tree                        140
Human                       138
background                  118
animals                     117
utensils                     65
Mythological characters      60
animal                       50
Tree                         49
hut                          44
Hut                          31
Animals                      27
flower                       26
birds                        25
river                        17
Flower                       10
Boat                          9
person                        9
bird                          7
boat                          7
room                          6
Bird                          5
Birds                         3
humans                        3
woman                         2
chariot                       2
humm                          2
River                         2
mythological character        2
Name: labels, dtype: int64

In [33]:
def rename_label(s):
    if s=='tree' or s=='Tree':
        return 'Tree'
    else:
        return 'Not Tree'

In [34]:
combined_df['labels']=combined_df['labels'].apply(rename_label)

In [35]:
combined_df

,labels,features,annotator,image_number,folder_name
0,Not Tree,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
1,Not Tree,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
2,Not Tree,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
3,Not Tree,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
4,Not Tree,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
...,...,...,...,...,...
2659,Tree,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13
2660,Tree,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13
2661,Tree,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13
2662,Not Tree,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13


In [38]:
import pandas as pd
import numpy as np

def process_images_with_labels(df):
    # Group by unique image identifiers
    grouped = df.groupby(['folder_name', 'image_number'])

    processed_features = []
    labels = []
    feature_length=30000

      
    for group_identifier, group in grouped:
        
        if 'Tree' in group['labels'].values:
            
            animal_features = np.array([np.array(row) for row in group[group['labels'] == 'Tree']['features']])
            
            mean_features = np.mean(animal_features, axis=0)
            label = 'Tree'  
        else:
            
            mean_features = np.zeros(feature_length)  
            label = 'Not Tree'
        
        # Append the result and the label to the lists
        processed_features.append(mean_features)
        labels.append(label)

    # Return both the features and the labels
    return processed_features, labels

# Apply the function to the DataFrame
resulting_features, resulting_labels = process_images_with_labels(combined_df)

# Create a new DataFrame with the results and labels
result_df = pd.DataFrame({
    'mean_tree_features': resulting_features,
    'label': resulting_labels
})



In [40]:
def expand_features(row):
    return pd.Series(row['mean_tree_features'])

expanded_df = result_df.apply(expand_features, axis=1)
new_columns = [f"feature {i}" for i in range(1, 30001)]
expanded_df.columns = new_columns
tree_df = pd.concat([result_df, expanded_df], axis=1)

tree_df.drop(columns=['mean_tree_features'],inplace=True)

In [41]:
tree_df

,label,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 29991,feature 29992,feature 29993,feature 29994,feature 29995,feature 29996,feature 29997,feature 29998,feature 29999,feature 30000
0,Not Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Not Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Not Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Not Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Not Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,Not Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,Not Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,Not Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
tree_df.to_csv('tree.csv', index=False)


##### mythological creature

In [72]:
combined_df['labels'].value_counts()[:30]

human                      1299
mythological characters     343
tree                        140
Human                       138
background                  118
animals                     117
utensils                     65
Mythological characters      60
animal                       50
Tree                         49
hut                          44
Hut                          31
Animals                      27
flower                       26
birds                        25
river                        17
Flower                       10
Boat                          9
person                        9
bird                          7
boat                          7
room                          6
Bird                          5
Birds                         3
humans                        3
woman                         2
chariot                       2
humm                          2
River                         2
mythological character        2
Name: labels, dtype: int64

In [73]:
def rename_label(s):
    if s=='mythological characters' or s=='Mythological characters' or s=='mythological character':
        return 'Mythological creature'
    else:
        return 'Not mythological creature'

In [74]:
combined_df['labels']=combined_df['labels'].apply(rename_label)

In [76]:
import pandas as pd
import numpy as np

def process_images_with_labels(df):
    # Group by unique image identifiers
    grouped = df.groupby(['folder_name', 'image_number'])

    processed_features = []
    labels = []
    feature_length=30000

      
    for group_identifier, group in grouped:
        
        if 'Mythological creature' in group['labels'].values:
            
            animal_features = np.array([np.array(row) for row in group[group['labels'] == 'Mythological creature']['features']])
            
            mean_features = np.mean(animal_features, axis=0)
            label = 'Mythological creature'  
        else:
            
            mean_features = np.zeros(feature_length)  
            label = 'Not mythological creature'
        
        # Append the result and the label to the lists
        processed_features.append(mean_features)
        labels.append(label)

    # Return both the features and the labels
    return processed_features, labels

# Apply the function to the DataFrame
resulting_features, resulting_labels = process_images_with_labels(combined_df)

# Create a new DataFrame with the results and labels
result_df = pd.DataFrame({
    'mean_mythological_features': resulting_features,
    'label': resulting_labels
})



In [77]:
def expand_features(row):
    return pd.Series(row['mean_mythological_features'])

expanded_df = result_df.apply(expand_features, axis=1)
new_columns = [f"feature {i}" for i in range(1, 30001)]
expanded_df.columns = new_columns
mythological_df = pd.concat([result_df, expanded_df], axis=1)

mythological_df.drop(columns=['mean_mythological_features'],inplace=True)

In [79]:
mythological_df.to_csv('mythological.csv',index=False)

In [80]:
combined_df

,labels,features,annotator,image_number,folder_name
0,Mythological creature,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
1,Mythological creature,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
2,Mythological creature,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
3,Not mythological creature,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
4,Not mythological creature,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s1.jpg,s1
...,...,...,...,...,...
2659,Not mythological creature,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13
2660,Not mythological creature,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13
2661,Not mythological creature,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13
2662,Not mythological creature,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",C:\Users\acer\OneDrive\Desktop\annotator\Task ...,s8.jpg,s13


In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('mean_animal.csv')

In [9]:
df['label']=df['label'].apply(modify_label)

In [6]:
def modify_label(r):
    if r=='Animal':
        return 1
    else:
        return 0

In [8]:
df

,label,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 29991,feature 29992,feature 29993,feature 29994,feature 29995,feature 29996,feature 29997,feature 29998,feature 29999,feature 30000
0,Animal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Not Animal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Animal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Animal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Not Animal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.984375,1.437500,1.359375,1.062500,1.640625,1.546875,1.171875,0.453125,0.453125,0.421875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Animal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
102,Not Animal,0.2,0.266667,0.4,2.8,2.6,1.8,4.466667,4.066667,2.533333,...,2.066667,3.866667,3.400000,2.333333,3.800000,3.400000,2.400000,0.333333,0.533333,0.333333
103,Animal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
104,Animal,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
df.to_csv('avg_animal.csv',index=False)